In [1]:
from random import sample
import random
from tqdm import tqdm
import pandas as pd
import requests
import time
import DHB as dhb
tqdm.pandas()

In [2]:
dhb.db_open()
df = dhb.sql("select * from match_aram")
dhb.db_close()

OPEN
close


In [3]:
df

,GAMEID,PARTICIPANTID,CHAMPIONID,CHAMPIONNAME,SPELL1,SPELL2,MAIN_RUNE,SUB_RUNE,STAT_RUNE,TEAMID,...,SELFMITIGATEDDAMAGE,TOTALTIMECCDEALT,TIMECCINGOTHERS,LARGESTKILLINGSPREE,GOLDEARNED,STARTING_ITEM,MAIN_ITEM_BUILD,SUB_ITEM_BUILD,BOOTS,SKILL_BUILD
0,KR_5656757004,1,12,Alistar,4,32,8400|8439|8463|8473|8451,8300|8304|8347,5002|5003|5005,100,...,87683,336,88,2,12546,1006|1006|1006|2051,3083|3001|3193,0,3111,1|2|3|1|1|4|1|2|1|2|4|2|2|3|3|4|3|3
1,KR_5656757004,2,222,Jinx,1,4,8000|8008|8009|9103|8017,8100|8139|8135,5002|5008|5005,100,...,11794,56,15,2,13818,1001|2031|3184,6672|3085|3031,0,3006,1|2|3|1|1|4|1|2|1|2|4|2|2|3|3|4|3|3
2,KR_5656757004,3,267,Nami,4,3,8100|8112|8126|8138|8106,8000|8009|8014,5003|5002|5008,100,...,9110,298,48,2,13061,2003|3070|3112,6617|3504|3011,3003,3158,1|2|3|2|2|4|2|3|2|3|4|1|1|3|3|4|1|1
3,KR_5656757004,4,268,Azir,4,1,8200|8229|8226|8210|8236,8000|8009|8014,5002|5008|5005,100,...,11828,190,14,6,16570,3802,6653|3115|3135,3102|3165,3020,1|3|1|1|4|1|2|1|2|4|2|2|3|3|4|3|3
4,KR_5656757004,5,121,Khazix,32,4,8100|8128|8126|8138|8135,8000|8009|8014,5002|5008|5008,100,...,21103,256,22,3,14117,3070|3177,6691|6693|3004,3142|6691,3111,1|2|3|2|2|4|2|2|1|2|1|4|1|1|1|3|3|4|3|3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338055,KR_5659827693,6,126,Jayce,21,4,8100|8128|8126|8138|8135,8000|8009|8014,5001|5008|5007,200,...,15943,141,9,4,14555,1036|1036|2031|3070,3004|6692|6609,6694,3158,1|2|3|1|1|3|1|3|1|3|1|3|3|2|2|2|2|2
338056,KR_5659827693,7,53,Blitzcrank,32,4,8100|8128|8139|8138|8135,8200|8226|8237,5002|5008|5008,200,...,21344,195,68,2,14055,2003|2003|3802,6656|4629|4645,3102,3158,1|2|3|1|1|4|1|3|1|3|4|3|3|2|2|4|2|2
338057,KR_5659827693,8,117,Lulu,3,4,8200|8214|8226|8210|8237,8000|8009|8014,5002|5008|5008,200,...,8127,213,34,2,13004,3070|4642,2065|3011|6616,3003,3158,1|2|3|1|1|4|1|3|1|3|4|3|3|2|2|4|2|2
338058,KR_5659827693,9,777,Yone,4,32,8000|8008|9111|9104|8014,8100|8139|8135,5002|5008|5005,200,...,32015,68,30,4,16533,6670,6673|3033|3031,3091,3006,1|2|3|1|1|4|1|2|1|2|4|2|2|3|3|4|3|3


In [342]:
df.columns

Index(['GAMEID', 'PARTICIPANTID', 'CHAMPIONID', 'CHAMPIONNAME', 'SPELL1',
       'SPELL2', 'MAIN_RUNE', 'SUB_RUNE', 'STAT_RUNE', 'TEAMID', 'WIN',
       'KILLS', 'DEATHS', 'ASSISTS', 'TOTALHEAL', 'TOTALDAMAGETAKEN',
       'TOTALDAMAGEDEALTTOCHAMPIONS', 'PHYSICALDAMAGEDEALTTOCHAMPIONS',
       'MAGICDAMAGEDEALTTOCHAMPIONS', 'TOTALDAMAGESHIELDEDONTEAMMATES',
       'TOTALHEALSONTEAMMATES', 'SELFMITIGATEDDAMAGE', 'TOTALTIMECCDEALT',
       'TIMECCINGOTHERS', 'LARGESTKILLINGSPREE', 'GOLDEARNED', 'STARTING_ITEM',
       'MAIN_ITEM_BUILD', 'SUB_ITEM_BUILD', 'BOOTS', 'SKILL_BUILD'],
      dtype='object')

In [344]:
df = df.groupby(["GAMEID"])[['PARTICIPANTID']].count()

In [345]:
df[df.PARTICIPANTID != 10]

,PARTICIPANTID
GAMEID,
KR_5633731638,9
KR_5647071237,9
KR_5650163748,9
KR_5650207239,9
KR_5650276617,9
KR_5653987993,9
KR_5655893014,9
KR_5658138764,9
KR_5658456701,9


In [325]:
df.isnull().sum()

PARTICIPANTID    0
dtype: int64

In [7]:
df["WIN"] = df.apply(lambda x: 1 if x["WIN"] == "True" else 0 , axis=1)

# 1.

In [16]:
def rune_img(row):
    tmp = []
    tmp += list(map(lambda x: 'https://ddragon.leagueoflegends.com/cdn/img/'+rune_lst[x], row["MAIN_RUNE"].split("|")))
    tmp += list(map(lambda x: 'https://ddragon.leagueoflegends.com/cdn/img/'+rune_lst[x], row["SUB_RUNE"].split("|")))
    tmp += list(map(lambda x: stat_rune[x], row["STAT_RUNE"].split("|")))
    tmp += [str(row.RATE)]
    
    tmp = "|".join(tmp)
    return tmp

In [160]:
# 먼저 메인룬만 뽑아옴
url = "https://ddragon.leagueoflegends.com/cdn/11.24.1/data/ko_KR/runesReforged.json"
res = requests.get(url).json()
main_rune = list(map(lambda x: ({str(x["id"]):x["icon"]}), res))
main_rune = {**main_rune[0],**main_rune[1],**main_rune[2],**main_rune[3],**main_rune[4]}

# 서브룬을 만들어 줌
sub_rune = list(map(lambda x: list(map(lambda z: list(map(lambda c: ({str(c["id"]):c["icon"]}), z["runes"])), x["slots"])), res))

# 3차원배열이여서 1차원으로 축소
sub_rune = [element for array in sub_rune for element in array]
sub_rune = [element for array in sub_rune for element in array]

# key값으로 검색할 수 있게 하나의 딕셔너리로 만드는 과정
sub_rune_lst = {**sub_rune[0],**sub_rune[1],**sub_rune[2],**sub_rune[3],**sub_rune[4],**sub_rune[5],**sub_rune[6]
               ,**sub_rune[7],**sub_rune[8],**sub_rune[9],**sub_rune[10],**sub_rune[11],**sub_rune[12],**sub_rune[13]
               ,**sub_rune[14],**sub_rune[15],**sub_rune[16],**sub_rune[17],**sub_rune[18],**sub_rune[19],**sub_rune[20]
               ,**sub_rune[21],**sub_rune[22],**sub_rune[23],**sub_rune[24],**sub_rune[25],**sub_rune[26],**sub_rune[27]
               ,**sub_rune[28],**sub_rune[29],**sub_rune[30],**sub_rune[31],**sub_rune[32],**sub_rune[33],**sub_rune[34]
               ,**sub_rune[35],**sub_rune[36],**sub_rune[37],**sub_rune[38],**sub_rune[39],**sub_rune[40],**sub_rune[41]
               ,**sub_rune[42],**sub_rune[43],**sub_rune[44],**sub_rune[45],**sub_rune[46],**sub_rune[47],**sub_rune[48]
               ,**sub_rune[49],**sub_rune[50],**sub_rune[51],**sub_rune[52],**sub_rune[53],**sub_rune[54],**sub_rune[55]
               ,**sub_rune[56],**sub_rune[57],**sub_rune[58],**sub_rune[59],**sub_rune[60],**sub_rune[61],**sub_rune[62]}

# main 룬과 sub룬을 합침 -? rune_lst 룬id : 이미지 링크 매핑 -> 키값만 넣으면 바로 링크가 뽑힘.
rune_lst = {**main_rune, **sub_rune_lst}

# stat 룬 이미지는 해외 칼바람 사이트에서 사용하는 api를 가져옴
stat_rune = {"5008":'https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodsadaptiveforceicon.png',
"5002":'https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodsarmoricon.png',
"5005":'https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodsattackspeedicon.png',
"5007":'https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodscdrscalingicon.png',
"5001":'https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodshealthscalingicon.png',
"5003":'https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodsmagicresicon.png'}

# champ_lst 를 가져옴. 형태는 id:champname 인데 나중에 변경할 가능성 있음.
# url = 'http://ddragon.leagueoflegends.com/cdn/11.23.1/data/en_US/champion.json'
# res = requests.get(url).json()
# champ_lst = list(map(lambda x : (res['data'][x]['key'],x), list(res['data'].keys())))
# champ_lst = dict(champ_lst)
# champ_lst = list(champ_lst.values())
# champ_lst[29] = "FiddleSticks" # 반드시 해주어야 함. (match에서 받아오는 피들스틱은 s가 대문자이기 때문)
champ_lst = list(df.groupby("CHAMPIONNAME").count().reset_index()["CHAMPIONNAME"])

In [12]:
def test(df):
    # test는 임시변수명, 나중에 수정할 것
    test = df.groupby(["CHAMPIONNAME","MAIN_RUNE","SUB_RUNE","STAT_RUNE"])[["WIN"]].count().rename(columns={"WIN":"GAMES"})
    test.sort_values("GAMES", ascending=False, inplace=True)
    test.reset_index(inplace=True)

    test1 = df.groupby(["CHAMPIONNAME","MAIN_RUNE","SUB_RUNE","STAT_RUNE"])[["WIN"]].sum()
    test1.sort_values("WIN", ascending=False, inplace=True)
    test1.reset_index(inplace=True)

    test = pd.merge(test,test1, on=["CHAMPIONNAME","MAIN_RUNE","SUB_RUNE","STAT_RUNE"])
    test["rate"] = round(test.WIN / test.GAMES * 100)
    
    tmp = []
    for i in champ_lst:
        tmp_lst = []
        tmp_lst.append(test[test.CHAMPIONNAME == i].iloc[0]["CHAMPIONNAME"])
        tmp_lst.append(test[test.CHAMPIONNAME == i].iloc[0]["MAIN_RUNE"])
        tmp_lst.append(test[test.CHAMPIONNAME == i].iloc[0]["SUB_RUNE"])
        tmp_lst.append(test[test.CHAMPIONNAME == i].iloc[0]["STAT_RUNE"])
        tmp_lst.append(test[test.CHAMPIONNAME == i].iloc[0]["rate"])
        tmp.append(tmp_lst)

    #     베스트 룬 탑2를 할때 밑에를 추가 함. 딱 젤 많이 가는걸로 보여줄지는 생각해볼 필요가 있음
    #     tmp_lst.append(test[test.CHAMPIONNAME == i].iloc[1]["CHAMPIONNAME"])
    #     tmp_lst.append(test[test.CHAMPIONNAME == i].iloc[1]["MAIN_RUNE"])
    #     tmp_lst.append(test[test.CHAMPIONNAME == i].iloc[1]["SUB_RUNE"])
    #     tmp_lst.append(test[test.CHAMPIONNAME == i].iloc[1]["STAT_RUNE"])
    #     tmp_lst.append(test[test.CHAMPIONNAME == i].iloc[1]["rate"])
    #     tmp.append(tmp_lst)
    PD = pd.DataFrame(tmp, columns=["CHAMPIONNAME","MAIN_RUNE","SUB_RUNE","STAT_RUNE","RATE"])

    PD["rune_img"] = PD.apply(lambda x: rune_img(x) ,axis=1)
    
    return PD

In [17]:
PD = test(df)

In [20]:
PD

,CHAMPIONNAME,MAIN_RUNE,SUB_RUNE,STAT_RUNE,RATE,rune_img
0,Aatrox,8000|8010|9111|9105|8299,8100|8139|8135,5002|5008|5008,50.0,https://ddragon.leagueoflegends.com/cdn/img/pe...
1,Ahri,8100|8112|8143|8138|8106,8200|8210|8226,5002|5008|5008,46.0,https://ddragon.leagueoflegends.com/cdn/img/pe...
2,Akali,8000|8010|8009|9105|8014,8100|8139|8135,5002|5008|5008,40.0,https://ddragon.leagueoflegends.com/cdn/img/pe...
3,Akshan,8000|8005|8009|9104|8014,8100|8139|8135,5002|5008|5005,35.0,https://ddragon.leagueoflegends.com/cdn/img/pe...
4,Alistar,8400|8439|8463|8473|8451,8000|9105|9111,5001|5002|5007,56.0,https://ddragon.leagueoflegends.com/cdn/img/pe...
...,...,...,...,...,...,...
152,Zed,8100|8112|8143|8138|8106,8000|8009|8014,5002|5008|5008,45.0,https://ddragon.leagueoflegends.com/cdn/img/pe...
153,Ziggs,8100|8128|8126|8138|8106,8000|8009|8014,5002|5008|5008,56.0,https://ddragon.leagueoflegends.com/cdn/img/pe...
154,Zilean,8100|8128|8126|8138|8106,8000|8009|8014,5002|5008|5008,38.0,https://ddragon.leagueoflegends.com/cdn/img/pe...
155,Zoe,8100|8128|8139|8138|8135,8000|8009|8014,5002|5008|5008,44.0,https://ddragon.leagueoflegends.com/cdn/img/pe...


In [8]:
champ_games = df.groupby("CHAMPIONNAME")[["GAMEID"]].count().rename(columns = {"GAMEID":"GAMES"}).reset_index()

In [24]:
champ_games

,CHAMPIONNAME,GAMES
0,Aatrox,2919
1,Ahri,2432
2,Akali,3077
3,Akshan,2047
4,Alistar,2104
...,...,...
152,Zed,3761
153,Ziggs,2775
154,Zilean,2085
155,Zoe,2719


In [63]:
rune_games = df.groupby(["CHAMPIONNAME","MAIN_RUNE","SUB_RUNE","STAT_RUNE"])[["WIN"]].count().rename(columns={"WIN":"RUNE_GAMES"}).reset_index()
rune_win = df.groupby(["CHAMPIONNAME","MAIN_RUNE","SUB_RUNE","STAT_RUNE"])[["WIN"]].sum().reset_index()
rune_win_pick = pd.merge(pd.merge(rune_games, rune_win, on=["CHAMPIONNAME","MAIN_RUNE","SUB_RUNE","STAT_RUNE"]),champ_games, on="CHAMPIONNAME")
rune_win_pick["PICK_RATE"] = round(rune_win_pick.RUNE_GAMES / rune_win_pick.GAMES * 100,1)
rune_win_pick["WIN_RATE"] = round(rune_win_pick.WIN / rune_win_pick.RUNE_GAMES * 100,1)
rune_win_pick = rune_win_pick.sort_values("PICK_RATE", ascending=False)

In [64]:
rune_win_pick

,CHAMPIONNAME,MAIN_RUNE,SUB_RUNE,STAT_RUNE,RUNE_GAMES,WIN,GAMES,PICK_RATE,WIN_RATE
119692,Vex,8100|8128|8126|8138|8106,8000|8009|8014,5002|5008|5008,495,242,2192,22.6,48.9
105453,Taliyah,8100|8128|8126|8138|8135,8000|8009|8014,5002|5008|5008,252,131,1127,22.4,52.0
62453,Lux,8100|8128|8126|8138|8106,8000|8009|8014,5002|5008|5008,775,378,3695,21.0,48.8
74492,Neeko,8100|8128|8126|8138|8106,8000|8009|8014,5002|5008|5008,253,108,1305,19.4,42.7
43954,Jinx,8000|8008|8009|9103|8014,8100|8139|8135,5002|5008|5005,791,431,4155,19.0,54.5
...,...,...,...,...,...,...,...,...,...
33356,Graves,8000|8021|8009|9103|8014,8100|8106|8143,5003|5008|5007,1,0,3653,0.0,0.0
81862,Pyke,8100|8128|8143|8138|8106,8400|8453|8444,5002|5008|5008,1,1,3783,0.0,100.0
81863,Pyke,8100|8128|8143|8138|8106,8400|8453|8473,5002|5008|5008,1,0,3783,0.0,0.0
81864,Pyke,8100|8128|8143|8138|8106,8400|8463|8444,5001|5008|5008,1,1,3783,0.0,100.0


In [38]:
rune_win_pick.groupby("CHAMPIONNAME")[["PICK_RATE"]].sum()

,PICK_RATE
CHAMPIONNAME,
Aatrox,82.2
Ahri,73.7
Akali,86.3
Akshan,76.0
Alistar,58.0
...,...
Zed,90.1
Ziggs,85.3
Zilean,68.8


#  과정

In [115]:
rune = df.iloc[0]["SUB_RUNE"].split("|")

In [116]:
rune

['8300', '8304', '8347']

In [114]:
list(map(lambda x: 'https://ddragon.leagueoflegends.com/cdn/img/'+rune_lst[x], rune))

['https://ddragon.leagueoflegends.com/cdn/img/perk-images/Styles/7204_Resolve.png',
 'https://ddragon.leagueoflegends.com/cdn/img/perk-images/Styles/Resolve/VeteranAftershock/VeteranAftershock.png',
 'https://ddragon.leagueoflegends.com/cdn/img/perk-images/Styles/Resolve/FontOfLife/FontOfLife.png',
 'https://ddragon.leagueoflegends.com/cdn/img/perk-images/Styles/Resolve/BonePlating/BonePlating.png',
 'https://ddragon.leagueoflegends.com/cdn/img/perk-images/Styles/Resolve/Overgrowth/Overgrowth.png']

In [117]:
list(map(lambda x: 'https://ddragon.leagueoflegends.com/cdn/img/'+rune_lst[x], rune))

['https://ddragon.leagueoflegends.com/cdn/img/perk-images/Styles/7203_Whimsy.png',
 'https://ddragon.leagueoflegends.com/cdn/img/perk-images/Styles/Inspiration/MagicalFootwear/MagicalFootwear.png',
 'https://ddragon.leagueoflegends.com/cdn/img/perk-images/Styles/Inspiration/CosmicInsight/CosmicInsight.png']

위에 있는 두개의 list(map())으로 메인룬, 서브룬의 이미지 url을 가져올 수 있음.

- 적응형 - 5008
- 공속 - 5005
- 쿨감 - 5007

- 체력 - 5001
- 방어 - 5002
- 마저 - 5003

In [130]:
stat_rune = {"5008":'https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodsadaptiveforceicon.png',
"5002":'https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodsarmoricon.png',
"5005":'https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodsattackspeedicon.png',
"5007":'https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodscdrscalingicon.png',
"5001":'https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodshealthscalingicon.png',
"5003":'https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodsmagicresicon.png'}

In [131]:
# stat_perk id : 이미지 링크 매핑
stat_rune

{'5008': 'https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodsadaptiveforceicon.png',
 '5002': 'https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodsarmoricon.png',
 '5005': 'https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodsattackspeedicon.png',
 '5007': 'https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodscdrscalingicon.png',
 '5001': 'https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodshealthscalingicon.png',
 '5003': 'https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodsmagicresicon.png'}

In [120]:
stat_perk = df.iloc[0]["STAT_RUNE"].split("|")

In [132]:
test = list(map(lambda x: stat_rune[x], stat_perk))

In [133]:
test

['https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodsarmoricon.png',
 'https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodsmagicresicon.png',
 'https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodsattackspeedicon.png']

### --------------------------------------------------------------------------------------------------------------------------------------------------------

In [338]:
test = df.groupby(["CHAMPIONNAME","MAIN_RUNE","SUB_RUNE","STAT_RUNE"])[["WIN"]].count().rename(columns={"WIN":"GAMES"})
test.sort_values("GAMES", ascending=False, inplace=True)
test.reset_index(inplace=True)
test1 = df.groupby(["CHAMPIONNAME","MAIN_RUNE","SUB_RUNE","STAT_RUNE"])[["WIN"]].sum()
test1.sort_values("WIN", ascending=False, inplace=True)
test1.reset_index(inplace=True)
test = pd.merge(test,test1, on=["CHAMPIONNAME","MAIN_RUNE","SUB_RUNE","STAT_RUNE"])
test["rate"] = round(test.WIN / test.GAMES * 100)

In [339]:
test

,CHAMPIONNAME,MAIN_RUNE,SUB_RUNE,STAT_RUNE,GAMES,WIN,rate
0,Jinx,8000|8008|8009|9103|8014,8100|8139|8135,5002|5008|5005,149,70,47.0
1,Lux,8100|8128|8126|8138|8106,8000|8009|8014,5002|5008|5008,123,55,45.0
2,Caitlyn,8000|8008|8009|9103|8014,8100|8139|8135,5002|5008|5005,110,55,50.0
3,MissFortune,8100|8128|8126|8138|8106,8000|8009|8014,5002|5008|5008,108,58,54.0
4,Varus,8100|8128|8139|8138|8135,8000|8009|8014,5002|5008|5008,107,62,58.0
...,...,...,...,...,...,...,...
36498,Kassadin,8100|8128|8143|8138|8134,8000|8009|8014,5003|5008|5008,1,0,0.0
36499,Kassadin,8100|8128|8143|8138|8106,8200|8226|8210,5003|5008|5008,1,0,0.0
36500,Kassadin,8100|8128|8143|8138|8106,8200|8226|8210,5003|5002|5007,1,1,100.0
36501,Kassadin,8100|8128|8143|8138|8106,8200|8226|8210,5002|5008|5008,1,0,0.0


In [167]:
url = 'http://ddragon.leagueoflegends.com/cdn/11.23.1/data/en_US/champion.json'
res = requests.get(url).json()
champ_lst = list(map(lambda x : (res['data'][x]['key'],x), list(res['data'].keys())))
champ_lst = dict(champ_lst)
champ_lst = list(champ_lst.values())
champ_lst[29] = "FiddleSticks"

In [283]:
tmp = []
for i in champ_lst:
    tmp_lst = []
    tmp_lst.append(test[test.CHAMPIONNAME == i].iloc[0]["CHAMPIONNAME"])
    tmp_lst.append(test[test.CHAMPIONNAME == i].iloc[0]["MAIN_RUNE"])
    tmp_lst.append(test[test.CHAMPIONNAME == i].iloc[0]["SUB_RUNE"])
    tmp_lst.append(test[test.CHAMPIONNAME == i].iloc[0]["STAT_RUNE"])
    tmp_lst.append(test[test.CHAMPIONNAME == i].iloc[0]["rate"])
    tmp.append(tmp_lst)

#     베스트 룬 탑2를 할때 밑에를 추가 함. 딱 젤 많이 가는걸로 보여줄지는 생각해볼 필요가 있음
#     tmp_lst = []
#     tmp_lst.append(test[test.CHAMPIONNAME == i].iloc[1]["CHAMPIONNAME"])
#     tmp_lst.append(test[test.CHAMPIONNAME == i].iloc[1]["MAIN_RUNE"])
#     tmp_lst.append(test[test.CHAMPIONNAME == i].iloc[1]["SUB_RUNE"])
#     tmp_lst.append(test[test.CHAMPIONNAME == i].iloc[1]["STAT_RUNE"])
#     tmp_lst.append(test[test.CHAMPIONNAME == i].iloc[1]["rate"])
#     tmp.append(tmp_lst)

In [284]:
PD = pd.DataFrame(tmp, columns=["CHAMPIONNAME","MAIN_RUNE","SUB_RUNE","STAT_RUNE","RATE"])

In [308]:
def rune_img(row):
    tmp = []
    tmp += list(map(lambda x: 'https://ddragon.leagueoflegends.com/cdn/img/'+rune_lst[x], row["MAIN_RUNE"].split("|")))
    tmp += list(map(lambda x: 'https://ddragon.leagueoflegends.com/cdn/img/'+rune_lst[x], row["SUB_RUNE"].split("|")))
    tmp += list(map(lambda x: stat_rune[x], row["STAT_RUNE"].split("|")))
    # tmp += str(int(row["RATE"])) 넣으면 숫자가 한자리씩 |가 들어감.. 일단 보류
    tmp = "|".join(tmp)
    return tmp

In [309]:
PD["rune_img"] = PD.apply(lambda x: rune_img(x) ,axis=1)

In [311]:
PD

,CHAMPIONNAME,MAIN_RUNE,SUB_RUNE,STAT_RUNE,RATE,rune_img
0,Aatrox,8000|8010|9111|9105|8299,8100|8139|8135,5002|5008|5008,45.0,https://ddragon.leagueoflegends.com/cdn/img/pe...
1,Ahri,8100|8112|8126|8138|8106,8000|8009|8014,5002|5008|5008,50.0,https://ddragon.leagueoflegends.com/cdn/img/pe...
2,Akali,8000|8010|8009|9105|8014,8100|8139|8135,5002|5008|5008,37.0,https://ddragon.leagueoflegends.com/cdn/img/pe...
3,Akshan,8000|8005|8009|9104|8014,8100|8139|8135,5002|5008|5005,24.0,https://ddragon.leagueoflegends.com/cdn/img/pe...
4,Alistar,8400|8439|8463|8473|8451,8000|9105|9111,5001|5002|5007,78.0,https://ddragon.leagueoflegends.com/cdn/img/pe...
...,...,...,...,...,...,...
152,Zed,8100|8112|8143|8138|8106,8000|8009|8014,5002|5008|5008,39.0,https://ddragon.leagueoflegends.com/cdn/img/pe...
153,Ziggs,8100|8128|8126|8138|8106,8000|8009|8014,5002|5008|5008,61.0,https://ddragon.leagueoflegends.com/cdn/img/pe...
154,Zilean,8100|8128|8126|8138|8106,8000|8009|8014,5002|5008|5008,42.0,https://ddragon.leagueoflegends.com/cdn/img/pe...
155,Zoe,8100|8128|8139|8138|8135,8000|8009|8014,5002|5008|5008,39.0,https://ddragon.leagueoflegends.com/cdn/img/pe...


# 2.

In [3]:
# 먼저 메인룬만 뽑아옴
url = "https://ddragon.leagueoflegends.com/cdn/11.24.1/data/ko_KR/runesReforged.json"
res = requests.get(url).json()
main_rune = list(map(lambda x: ({str(x["id"]):x["icon"]}), res))
main_rune = {**main_rune[0],**main_rune[1],**main_rune[2],**main_rune[3],**main_rune[4]}

# 서브룬을 만들어 줌
sub_rune = list(map(lambda x: list(map(lambda z: list(map(lambda c: ({str(c["id"]):c["icon"]}), z["runes"])), x["slots"])), res))

# 3차원배열이여서 1차원으로 축소
sub_rune = [element for array in sub_rune for element in array]
sub_rune = [element for array in sub_rune for element in array]

# key값으로 검색할 수 있게 하나의 딕셔너리로 만드는 과정
sub_rune_lst = {**sub_rune[0],**sub_rune[1],**sub_rune[2],**sub_rune[3],**sub_rune[4],**sub_rune[5],**sub_rune[6]
               ,**sub_rune[7],**sub_rune[8],**sub_rune[9],**sub_rune[10],**sub_rune[11],**sub_rune[12],**sub_rune[13]
               ,**sub_rune[14],**sub_rune[15],**sub_rune[16],**sub_rune[17],**sub_rune[18],**sub_rune[19],**sub_rune[20]
               ,**sub_rune[21],**sub_rune[22],**sub_rune[23],**sub_rune[24],**sub_rune[25],**sub_rune[26],**sub_rune[27]
               ,**sub_rune[28],**sub_rune[29],**sub_rune[30],**sub_rune[31],**sub_rune[32],**sub_rune[33],**sub_rune[34]
               ,**sub_rune[35],**sub_rune[36],**sub_rune[37],**sub_rune[38],**sub_rune[39],**sub_rune[40],**sub_rune[41]
               ,**sub_rune[42],**sub_rune[43],**sub_rune[44],**sub_rune[45],**sub_rune[46],**sub_rune[47],**sub_rune[48]
               ,**sub_rune[49],**sub_rune[50],**sub_rune[51],**sub_rune[52],**sub_rune[53],**sub_rune[54],**sub_rune[55]
               ,**sub_rune[56],**sub_rune[57],**sub_rune[58],**sub_rune[59],**sub_rune[60],**sub_rune[61],**sub_rune[62]}

# main 룬과 sub룬을 합침 -? rune_lst 룬id : 이미지 링크 매핑 -> 키값만 넣으면 바로 링크가 뽑힘.
rune_lst = {**main_rune, **sub_rune_lst}

# stat 룬 이미지는 해외 칼바람 사이트에서 사용하는 api를 가져옴
stat_rune = {"5008":'https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodsadaptiveforceicon.png',
"5002":'https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodsarmoricon.png',
"5005":'https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodsattackspeedicon.png',
"5007":'https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodscdrscalingicon.png',
"5001":'https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodshealthscalingicon.png',
"5003":'https://raw.communitydragon.org/11.24/plugins/rcp-be-lol-game-data/global/default/v1/perk-images/statmods/statmodsmagicresicon.png'}

# champ_lst 를 가져옴. 형태는 id:champname 인데 나중에 변경할 가능성 있음.
# url = 'http://ddragon.leagueoflegends.com/cdn/11.23.1/data/en_US/champion.json'
# res = requests.get(url).json()
# champ_lst = list(map(lambda x : (res['data'][x]['key'],x), list(res['data'].keys())))
# champ_lst = dict(champ_lst)
# champ_lst = list(champ_lst.values())
# champ_lst[29] = "FiddleSticks" # 반드시 해주어야 함. (match에서 받아오는 피들스틱은 s가 대문자이기 때문)
champ_lst = list(df.groupby("CHAMPIONNAME").count().reset_index()["CHAMPIONNAME"])

In [17]:
def champ_rune_build(df):
    
    df["MAIN"] = df.apply(lambda x: x["MAIN_RUNE"].split("|")[0], axis=1)
    df["SUB"] = df.apply(lambda x: x["SUB_RUNE"].split("|")[0], axis=1)
    
    rune_games = df.groupby(["CHAMPIONNAME","MAIN_RUNE","SUB_RUNE","STAT_RUNE"])[["WIN"]].count().rename(columns={"WIN":"RUNE_GAMES"}).reset_index()
    rune_win = df.groupby(["CHAMPIONNAME","MAIN_RUNE","SUB_RUNE","STAT_RUNE"])[["WIN"]].sum().reset_index()
    rune_win_pick = pd.merge(pd.merge(rune_games, rune_win, on=["CHAMPIONNAME","MAIN_RUNE","SUB_RUNE","STAT_RUNE"]),champ_games, on="CHAMPIONNAME")
    rune_win_pick["PICK_RATE"] = round(rune_win_pick.RUNE_GAMES / rune_win_pick.GAMES * 100,1)
    rune_win_pick["WIN_RATE"] = round(rune_win_pick.WIN / rune_win_pick.RUNE_GAMES * 100,1)
    rune_win_pick = rune_win_pick.sort_values("PICK_RATE", ascending=False)
    rune_win_pick["MAIN"] = rune_win_pick.apply(lambda x: x["MAIN_RUNE"].split("|")[0], axis=1)
    rune_win_pick["SUB"] = rune_win_pick.apply(lambda x: x["SUB_RUNE"].split("|")[0], axis=1)
    rune_build = rune_win_pick


    rune_games = df.groupby(["CHAMPIONNAME","MAIN","SUB"])[["WIN"]].count().rename(columns={"WIN":"RUNE_GAMES"}).reset_index()
    rune_win = df.groupby(["CHAMPIONNAME","MAIN","SUB"])[["WIN"]].sum().reset_index()
    rune_win_pick = pd.merge(pd.merge(rune_games, rune_win, on=["CHAMPIONNAME","MAIN","SUB"]),champ_games, on="CHAMPIONNAME")
    rune_win_pick["PICK_RATE"] = round(rune_win_pick.RUNE_GAMES / rune_win_pick.GAMES * 100,1)
    rune_win_pick["WIN_RATE"] = round(rune_win_pick.WIN / rune_win_pick.RUNE_GAMES * 100,1)
    rune_win_pick = rune_win_pick.sort_values("PICK_RATE", ascending=False)
    main_sub_rune = rune_win_pick[["CHAMPIONNAME","MAIN","SUB","PICK_RATE","WIN_RATE"]].rename(columns={"PICK_RATE":"MAIN_PICK_RATE","WIN_RATE":"MAIN_WIN_RATE"})

    rune = pd.merge(rune_build,main_sub_rune, on=["CHAMPIONNAME","MAIN","SUB"]).sort_values(["MAIN_PICK_RATE","PICK_RATE"], ascending=False)
    
    tmp = []
    for i in champ_lst:
        tmp_lst = []
        champ = rune[rune.CHAMPIONNAME == i]
        tmp_lst.append(champ.iloc[0]["CHAMPIONNAME"])
        tmp_lst.append(champ.iloc[0]["MAIN_RUNE"])
        tmp_lst.append(champ.iloc[0]["SUB_RUNE"])
        tmp_lst.append(champ.iloc[0]["STAT_RUNE"])
        tmp_lst.append(champ.iloc[0]["PICK_RATE"])
        tmp_lst.append(champ.iloc[0]["WIN_RATE"])
        tmp_lst.append(champ.iloc[0]["MAIN"])
        tmp_lst.append(champ.iloc[0]["SUB"])
        tmp_lst.append(champ.iloc[0]["MAIN_PICK_RATE"])
        tmp_lst.append(champ.iloc[0]["MAIN_WIN_RATE"])

        s_champ = champ[champ.MAIN_PICK_RATE < champ.iloc[0]["MAIN_PICK_RATE"]]
        tmp_lst.append(s_champ.iloc[0]["MAIN_RUNE"])
        tmp_lst.append(s_champ.iloc[0]["SUB_RUNE"])
        tmp_lst.append(s_champ.iloc[0]["STAT_RUNE"])
        tmp_lst.append(s_champ.iloc[0]["PICK_RATE"])
        tmp_lst.append(s_champ.iloc[0]["WIN_RATE"])
        tmp_lst.append(s_champ.iloc[0]["MAIN"])
        tmp_lst.append(s_champ.iloc[0]["SUB"])
        tmp_lst.append(s_champ.iloc[0]["MAIN_PICK_RATE"])
        tmp_lst.append(s_champ.iloc[0]["MAIN_WIN_RATE"])

        tmp.append(tmp_lst)

    champ_rune = pd.DataFrame(tmp,columns=["CHAMPIONNAME","F_MAIN_RUNE","F_SUB_RUNE","F_STAT_RUNE","F_PICK_RATE","F_WIN_RATE","F_MAIN","F_SUB",
                                           "F_MAIN_PICK_RATE","F_MAIN_WIN_RATE",
                                           "S_MAIN_RUNE","S_SUB_RUNE","S_STAT_RUNE","S_PICK_RATE","S_WIN_RATE","S_MAIN","S_SUB",
                                           "S_MAIN_PICK_RATE","S_MAIN_WIN_RATE"])

    champ_rune["RUNE_IMG_ONE"] = champ_rune.apply(lambda x: rune_img_one(x), axis=1)
    champ_rune["RUNE_IMG_TWO"] = champ_rune.apply(lambda x: rune_img_two(x), axis=1)

    return champ_rune

In [18]:
# 만들기는 했으나 너무나 길어서 함수를 두개 만들고 첫번째룬 컬럼, 두번째룬 컬럼으로 만들 고려중.
# 또한 표현할때 세부 룬의 픽률과 승률은 제외할지는 고려해야함. ( 지금은 뺀 상태 ) -> 해외 칼바람처럼 표기
def rune_img_one(row):
    tmp = []
    tmp += list(map(lambda x: 'https://ddragon.leagueoflegends.com/cdn/img/'+rune_lst[x], row["F_MAIN_RUNE"].split("|")))
    tmp += list(map(lambda x: 'https://ddragon.leagueoflegends.com/cdn/img/'+rune_lst[x], row["F_SUB_RUNE"].split("|")))
    tmp += list(map(lambda x: stat_rune[x], row["F_STAT_RUNE"].split("|")))
    tmp += [str(row.F_MAIN_PICK_RATE)]
    tmp += [str(row.F_MAIN_WIN_RATE)]
    
    tmp = "|".join(tmp)
    return tmp

In [19]:
def rune_img_two(row):
    tmp = []    
    tmp += list(map(lambda x: 'https://ddragon.leagueoflegends.com/cdn/img/'+rune_lst[x], row["S_MAIN_RUNE"].split("|")))
    tmp += list(map(lambda x: 'https://ddragon.leagueoflegends.com/cdn/img/'+rune_lst[x], row["S_SUB_RUNE"].split("|")))
    tmp += list(map(lambda x: stat_rune[x], row["S_STAT_RUNE"].split("|")))
    tmp += [str(row.S_MAIN_PICK_RATE)]
    tmp += [str(row.S_MAIN_WIN_RATE)]
    
    tmp = "|".join(tmp)
    return tmp

In [20]:
champ_rune = champ_rune_build(df)

In [23]:
champ_rune[["CHAMPIONNAME","RUNE_IMG_ONE","RUNE_IMG_TWO"]]

,CHAMPIONNAME,RUNE_IMG_ONE,RUNE_IMG_TWO
0,Aatrox,https://ddragon.leagueoflegends.com/cdn/img/pe...,https://ddragon.leagueoflegends.com/cdn/img/pe...
1,Ahri,https://ddragon.leagueoflegends.com/cdn/img/pe...,https://ddragon.leagueoflegends.com/cdn/img/pe...
2,Akali,https://ddragon.leagueoflegends.com/cdn/img/pe...,https://ddragon.leagueoflegends.com/cdn/img/pe...
3,Akshan,https://ddragon.leagueoflegends.com/cdn/img/pe...,https://ddragon.leagueoflegends.com/cdn/img/pe...
4,Alistar,https://ddragon.leagueoflegends.com/cdn/img/pe...,https://ddragon.leagueoflegends.com/cdn/img/pe...
...,...,...,...
152,Zed,https://ddragon.leagueoflegends.com/cdn/img/pe...,https://ddragon.leagueoflegends.com/cdn/img/pe...
153,Ziggs,https://ddragon.leagueoflegends.com/cdn/img/pe...,https://ddragon.leagueoflegends.com/cdn/img/pe...
154,Zilean,https://ddragon.leagueoflegends.com/cdn/img/pe...,https://ddragon.leagueoflegends.com/cdn/img/pe...
155,Zoe,https://ddragon.leagueoflegends.com/cdn/img/pe...,https://ddragon.leagueoflegends.com/cdn/img/pe...
